## This notebook is used to collect the Google Street View metadata
After generating the sample sites along the streets, you can use the coordinates of those site to generate the metadata of Google Street View images. The metadata includes the lon, lat, date information.

By Xiaojiang Li, Temple University, Feb 28, 2021

### Prepare the file and key
The input file is the created sample site shapefile, for the Google Street View API key, register here
https://developers.google.com/maps/documentation/streetview/metadata

In [4]:
import os, os.path
inputshp='../sample-spatialdata/Cambridge20m.shp'
outdir='metadata'
key=''
num=1000 # the batch size

if not os.path.exists(outdir):
    os.mkdir(outdir)

In [46]:
import ogr, osr
import time
import os,os.path
import sys
import json


if not os.path.exists(outdir):
    os.makedirs(outdir)

driver = ogr.GetDriverByName('ESRI Shapefile')

# change the projection of shapefile to the WGS84
dataset = driver.Open(inputshp)
layer = dataset.GetLayer()


# loop all the features in the featureclass
feature = layer.GetNextFeature()
featureNum = layer.GetFeatureCount()
# batch = featureNum/num    
batch = int(featureNum/num + 0.5)

print(batch)
for b in range(batch):
    # for each batch process num GSV site
    start = b*num
    end = (b+1)*num
    if end > featureNum:
        end = featureNum
    
    ouputTextFile = 'Pnt_start%s_end%s.txt'%(start,end)
    ouputGSVinfoFile = os.path.join(outdir,ouputTextFile)
    
    # skip over those existing txt files
    if os.path.exists(ouputGSVinfoFile):
        continue

    time.sleep(1)
    
    with open(ouputGSVinfoFile, 'w') as panoInfoText:
        # process num feature each time
        for i in range(start, end):
            feature = layer.GetFeature(i)        
            geom = feature.GetGeometryRef()
            
            # trasform the current projection of input shapefile to WGS84
            #WGS84 is Earth centered, earth fixed terrestrial ref system
            lon = geom.GetX()
            lat = geom.GetY()
            print(lon, lat)
            
            # get the meta data of panoramas 
            # urlAddress = r'http://maps.google.com/cbk?output=xml&ll=%s,%s&key=%s'%(lat,lon,key)
            urlAddress = r'https://maps.googleapis.com/maps/api/streetview/metadata?size=600x300&location=%s,%s&heading=-45&pitch=42&fov=110&key=%s'%(lat, lon, key)
            
            time.sleep(0.1)
            
            
            # using different url reading method in python2 and python3
            if sys.version_info[0] == 2:
                # from urllib2 import urlopen
                import urllib
                
                metaData = urllib.urlopen(urlAddress).read()

            if sys.version_info[0] == 3:
                import urllib.request

                request = urllib.request.Request(urlAddress)
                metaData = urllib.request.urlopen(request).read()
                
            
            data = json.loads(metaData)
            if data['status'] == 'NOT_FOUND': 
                print('The data is not existing')
                continue
                
            panoDate = data['date']
            panoId = data['pano_id']
            panoLat = data['location']['lat']
            panoLon = data['location']['lng']
            
            # print ('The coordinate (%s,%s), panoId is: %s, panoDate is: %s')%(panoLon,panoLat,panoId, panoDate)
            lineTxt = 'panoID: %s panoDate: %s longitude: %s latitude: %s\n'%(panoId, panoDate, panoLon, panoLat)
            panoInfoText.write(lineTxt)

    panoInfoText.close()
        

2
-122.25160692146233 37.800708139471986
-122.2514450147464 37.80064666705943
-122.25128779158693 37.80057804471926
-122.25113560961736 37.80050260886447
-122.25098540230803 37.80042481452175
-122.25083596943452 37.80034601096247
-122.25069126036819 37.800261908470766
-122.2505504465377 37.80017377844437
-122.25041457260876 37.800080907407285
-122.25028396031439 37.79998345183437
-122.25015994757838 37.799880761540486
-122.18277962875452 37.76606196310827
-122.182659668795 37.76595694039863
-122.1825482126292 37.7658455462881
-122.18243675646337 37.765734152009784
-122.18232411091682 37.76562350797008
-122.18221142424487 37.765512889711786
-122.18209873757293 37.76540227128806
-122.1819860418249 37.76529165847914
-122.18187330073194 37.76518107438358
-122.18176055963895 37.76507049012267
-122.18164781854598 37.76495990569641
-122.18153507745299 37.76484932110481
-122.18142233636003 37.764738736347844
-122.18130959526704 37.76462815142554
-122.18119685417408 37.764517566337894
-122.1810

KeyboardInterrupt: 